# Extract Key Information

In [0]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Set up Azure OpenAI

In [0]:
!pip install -r /Workspace/Repos/jeanpierre.vanniekerk@truenorthgroup.co.za/document-analysis-using-gpt-3.ide/notebooks/requirements.txt

  Using cached azure_identity-1.6.0-py2.py3-none-any.whl (108 kB)
  Using cached streamlit-1.18.1-py2.py3-none-any.whl (9.6 MB)
  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
  Using cached matplotlib-3.6.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.8 MB)
  Using cached plotly-5.12.0-py2.py3-none-any.whl (15.2 MB)
  Using cached scipy-1.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
  Using cached scikit_learn-1.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.5 MB)
  Using cached tiktoken-0.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached llama_index-0.4.33-py3-none-any.whl
  Using cached langchain-0.0.129-py3-none-any.whl (467 kB)
  Using cached faiss_cpu-1.7.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
  Using cached msal-1.23.0-py2.py3-none-any.whl (90 kB)
  Using cached msal_extensions-0.

In [0]:
import os
import openai
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

False

## Load Data

In [0]:
import pandas as pd
df = spark.sql('select * from openai.bbc_news_data').limit(10).toPandas()

## Request to API

In [0]:
# create prompt
prompt_prefix = """ 
  Extract key information from this text
"""

prompt = prompt_prefix + df['title'].loc[0] + "\n" + df['content'].loc[0]
print(prompt)

 
  Extract key information from this text
Ad sales boost Time Warner profit
" Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.  Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner in

In [0]:
# Request API
response = openai.Completion.create(
  deployment_id="text-davinci-003", 
  prompt=prompt,
  temperature=0,
  max_tokens=1000,
  top_p=0.95,
  frequency_penalty=1,
  presence_penalty=1
)

# print response
print(response['choices'][0]['text'])



Key Information: 
- Time Warner's quarterly profits increased 76% to $1.13bn for the three months to December, from $639m year-earlier
- Sales rose 2% to $11.1bn from $10.9bn in fourth quarter
- Profits were buoyed by one-off gains which offset a profit dip at Warner Bros and less users for AOL 
- Owns 8% of Google 
- AOL lost 464,000 subscribers in fourth quarter but underlying profit before exceptional items rose 8% on back of stronger internet advertising revenues 
- Film division saw profits slump 27%, helped by box office flops Alexander and Catwoman  
- Full year profits up 27%, revenue grew 6.4% to $42.09bn 
- Projecting operating earnings growth of around 5%, higher revenue and wider profit margins in 2005  
- To restate accounts as part of efforts to resolve inquiry into AOL by US market regulators; offered to pay $300m settlement


In [0]:
colname = 'key_info'
results = pd.DataFrame(columns=[colname], index=df.index)

prompt_prefix = """ 
  Extract key information from this text
"""

for idx, title, content in zip(df.index.values, df['title'].loc[df.index.values], df['content'].loc[df.index.values]):
  
  # build prompt
  prompt = prompt_prefix + title + "\n" + content

  try:
    # Request API
    response = openai.Completion.create(
      deployment_id="text-davinci-003", 
      prompt=prompt,
      temperature=0,
      max_tokens=1000,
      top_p=0.95,
      frequency_penalty=1,
      presence_penalty=1
    )

      # response
    results[colname].loc[idx] = response['choices'][0]['text']
  except Exception as err:
    idx
    print(f"Unexpected {err=}, {type(err)=}")

In [0]:
df_results = pd.concat([df, results], axis=1)
df_results.shape
df_results

(10, 6)

,category,filename,title,content,embedding,key_info
0,business,001.txt,Ad sales boost Time Warner profit,""" Quarterly profits at US media giant TimeWarn...","[-0.0012276918860152364, 0.00733763724565506, ...",\n\nKey Information: \n- Time Warner's quarter...
1,business,002.txt,Dollar gains on Greenspan speech,""" The dollar has hit its highest level against...","[0.0009311728645116091, 0.014099937863647938, ...",\n\nKey Information: \n- Dollar has hit highes...
2,business,003.txt,Yukos unit buyer faces loan claim,""" The owners of embattled Russian oil giant Yu...","[-0.010487922467291355, 0.009665092453360558, ...",\n\nKey Information: \n- Menatep Group is aski...
3,business,004.txt,High fuel prices hit BA's profits,""" British Airways has blamed high fuel prices ...","[0.0111119095236063, 0.004624682944267988, -0....",\n\nKey Information: \n- British Airways profi...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,"[-0.0021637482568621635, 0.005410161800682545,...",\n\nKey Information: \n- Pernod Ricard is cons...
5,business,006.txt,Japan narrowly escapes recession,""" Japan's economy teetered on the brink of a t...","[0.002582279033958912, 0.008218934759497643, -...",\n\nKey Information: \n- Japan's economy grew ...
6,business,007.txt,Jobs growth still slow in the US,""" The US created fewer jobs than expected in J...","[0.005617379676550627, 0.005877326242625713, -...","\n\nKey Information: \n- US firms added 146,00..."
7,business,008.txt,India calls for fair trade rules,""" India, which attends the G7 meeting of seven...","[0.00015200248162727803, 0.0035213909577578306...",\n\nKey Information: \n- India attends G7 meet...
8,business,009.txt,Ethiopia's crop production up 24%,""" Ethiopia produced 14.27 million tonnes of cr...","[0.008323440328240395, 0.014594526030123234, -...",\n\nKey Information: \n- Ethiopia produced 14....
9,business,010.txt,Court rejects $280bn tobacco case,A US government claim accusing the country's ...,"[-0.0033349236473441124, 0.014696489088237286,...",\n\nKey Information: \n- US government claim a...


## Save results

In [0]:
df_results = spark.createDataFrame(df_results)
df_results.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("openai.document_analysis_key_information")
df_results = spark.sql('select * from openai.document_analysis_predictions')
df_results.limit(10).toPandas()

,category,filename,title,content,embedding,n_tokens,prediction
0,1,232.txt,Connick Jr to lead Broadway show,""" Singer and actor Harry Connick Jr is to star...","[-0.0013730002101510763, -0.03857267275452614,...",254,1
1,1,236.txt,Franz Ferdinand's art school lesson,""" Scottish rock band Franz Ferdinand, who shot...","[-0.00918667670339346, 0.002770294900983572, 0...",694,1
2,1,238.txt,Portishead back after eight years,""" Cult British group Portishead have revealed ...","[-0.01821870543062687, -0.02119753137230873, -...",329,1
3,1,240.txt,Elvis 'set for chart hat-trick',The late US legend Elvis Presley is likely to...,"[-0.02780121937394142, -0.010392170399427414, ...",318,1
4,1,243.txt,Blue beat U2 to top France honour,""" Irish band U2 have been honoured at France's...","[-0.021403657272458076, -0.013215591199696064,...",249,1
5,1,244.txt,Ten-year tragedy of missing Manic,""" Richey Edwards, guitarist and lyricist for T...","[-0.014232789166271687, 0.020377222448587418, ...",829,1
6,1,250.txt,Prince crowned 'top music earner',""" Prince earned more than any other pop star i...","[0.0012800436234101653, -0.03181799128651619, ...",352,1
7,1,256.txt,Brits debate over 'urban' music,""" Joss Stone, a 17-year-old soul singer from D...","[-0.0016374018741771579, -0.010487286373972893...",2916,1
8,1,260.txt,Singer Christina Aguilera to wed,""" Pop star Christina Aguilera is to marry musi...","[-0.014513788744807243, -0.012519126757979393,...",256,1
9,1,264.txt,Grammys honour soul star Charles,""" The memory of soul legend Ray Charles domina...","[-0.013948743231594563, -0.005762143991887569,...",716,1


In [0]:
df_results.cache().count()

735